## Make the matricies from the raw data.

In [1]:
import pandas as pd
import numpy as np
from functions import create_matrix

In [2]:
Circos_DATA = pd.read_csv('Circos_DATA.csv')

### df structure

```
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Response ID                1434 non-null   object
 1   Code                       1434 non-null   int64 
 2   Weather/Climate            1434 non-null   int64 
 3   Hydrology                  1434 non-null   int64 
 4   Ecology                    1434 non-null   int64 
 5   Geomorphology              1434 non-null   int64 
 6   Demographic                1434 non-null   int64 
 7   Socioeconomic              1434 non-null   int64 
 8   Social Capital & Identity  1434 non-null   int64 
 9   Human Built Capital        1434 non-null   int64 
 10  Institutional              1434 non-null   int64 
 11  Livelihood activities      1434 non-null   int64 
 12  Commodity Prices           1434 non-null   int64 
 ```

In [3]:
df = create_matrix(Circos_DATA, code=0)
#print(df)

### Make the Plot (attempt using Plotly)

In [17]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from functions import check_data, get_ideogram_ends, map_data, make_ribbon_ends
PI = np.pi

In [5]:
# Determine the size of every arc in the ideogram, based on total number of manuscripts.
matrix = np.array(df, dtype=int)
L = check_data(matrix)

row_sum = [np.sum(matrix[k,:]) for k in range(L)]

#set the gap between two consecutive ideograms
gap=2*PI*0.005
ideogram_length=2*PI*np.asarray(row_sum)/sum(row_sum)-gap*np.ones(L)

# Get the end angular coordinates for each ideogram arc
ideo_ends = get_ideogram_ends(ideogram_length, gap)

In [6]:
# Labels and colors will be consistent across all figures
labels = list(df.columns)
ideo_colors=['rgba(244, 109, 67, 0.75)',
             'rgba(253, 174, 97, 0.75)',
             'rgba(254, 224, 139, 0.75)',
             'rgba(217, 239, 139, 0.75)',
             'rgba(166, 217, 106, 0.75)',
             'rgba(244, 109, 67, 0.75)',
             'rgba(253, 174, 97, 0.75)',
             'rgba(254, 224, 139, 0.75)',
             'rgba(217, 239, 139, 0.75)',
             'rgba(166, 217, 106, 0.75)'
              ]
# TODO: Use colorbrewer to get 10 nice colors.

In [7]:
mapped_data = map_data(matrix, row_sum, ideogram_length)
idx_sort=np.argsort(mapped_data, axis=1)
ribbon_ends=make_ribbon_ends(mapped_data, ideo_ends,  idx_sort)
ribbon_color=[L*[ideo_colors[k]] for k in range(L)]

### Create a Layout for the figure

In [8]:
from functions import make_layout, make_ribbon, make_self_rel, invPerm

layout=make_layout('Chord diagram', 400)

In [9]:
# these value are set after a few trials 
radii_sribb=[0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30, 0.30]

In [10]:
ribbon_info=[]
for k in range(L):

    sigma=idx_sort[k]
    sigma_inv=invPerm(sigma)
    for j in range(k, L):
        if matrix[k][j]==0 and matrix[j][k]==0: continue
        eta=idx_sort[j]
        eta_inv=invPerm(eta)
        l=ribbon_ends[k][sigma_inv[j]]

        if j==k:
            # CHANGE THIS APPEND TO A SIMPLE ADDITION....
            #layout['shapes'].append(make_self_rel(l, 'rgb(175,175,175)' ,
            #                        ideo_colors[k], radius=radii_sribb[k]))
            layout['shapes'] = layout['shapes'] + (make_self_rel(l, 'rgb(175,175,175)' ,
                                    ideo_colors[k], radius=radii_sribb[k]),)
            z=0.9*np.exp(1j*(l[0]+l[1])/2)
            #the text below will be displayed when hovering the mouse over the ribbon
            text='There were {:d}'.format(matrix[k][k])+' instances of manuscripts with {k} and {k}'.format(k=labels[k])
            ribbon_info.append(go.Scatter(x=[z.real],
                                       y=[z.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ideo_colors[k]),
                                       text=text,
                                       hoverinfo='text'
                                       )
                              )
        else:
            r=ribbon_ends[j][eta_inv[k]]
            zi=0.9*np.exp(1j*(l[0]+l[1])/2)
            zf=0.9*np.exp(1j*(r[0]+r[1])/2)
            #texti and textf are the strings that will be displayed when hovering the mouse 
            #over the two ribbon ends
            texti='There were {:d}'.format(matrix[k][j])+' instances of manuscripts with data on {k} and {j}'.format(
                k=labels[k], j=labels[j])
            textf='There were {:d}'.format(matrix[k][j])+' instances of manuscripts with data on {k} and {j}'.format(
                k=labels[k], j=labels[j])
            ribbon_info.append(go.Scatter(x=[zi.real],
                                       y=[zi.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ribbon_color[k][j]),
                                       text=texti,
                                       hoverinfo='text'
                                       )
                              ),
            ribbon_info.append(go.Scatter(x=[zf.real],
                                       y=[zf.imag],
                                       mode='markers',
                                       marker=dict(size=0.5, color=ribbon_color[k][j]),
                                       text=textf,
                                       hoverinfo='text'
                                       )
                              )
            r=(r[1], r[0])#IMPORTANT!!!  Reverse these arc ends because otherwise you get
                          # a twisted ribbon
            #append the ribbon shape
            layout['shapes'] = layout['shapes'] + (make_ribbon(l, r, 'rgb(175,175,175)' , ribbon_color[k][j]),)



In [ ]:
from functions import make_ideogram_arc, make_ideo_shape
ideograms=[]
for k in range(len(ideo_ends)):
    z= make_ideogram_arc(1.1, ideo_ends[k])
    zi=make_ideogram_arc(1.0, ideo_ends[k])
    m=len(z)
    n=len(zi)
    ideograms.append(go.Scatter(x=z.real,
                             y=z.imag,
                             mode='lines',
                             line=dict(color=ideo_colors[k], shape='spline', width=0.25),
                             text=labels[k]+'<br>'+'{:d}'.format(row_sum[k]),
                             hoverinfo='text'
                             )
                     )


    path='M '
    for s in range(m):
        path+=str(z.real[s])+', '+str(z.imag[s])+' L '

    Zi=np.array(zi.tolist()[::-1])

    for s in range(m):
        path+=str(Zi.real[s])+', '+str(Zi.imag[s])+' L '
    path+=str(z.real[0])+' ,'+str(z.imag[0])

    layout['shapes'] = layout['shapes'] + (make_ideo_shape(path,'rgb(150,150,150)' , ideo_colors[k]),)

data = go.Data(ideograms+ribbon_info)
fig = go.Figure(data=data, layout=layout)

#import plotly.offline as off
#off.init_notebook_mode()

py.iplot(fig, filename='chord-diagram-ses')


In [18]:
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode(connected=True)

data = go.Data(ribbon_info+ideograms)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='chord-diagram-ses')